In [2]:
import pandas as pd                                     # type: ignore
import numpy as np                                      # type: ignore
import warnings                                         # type: ignore
from sklearn.model_selection import train_test_split    # type: ignore
from sklearn.decomposition import PCA                   # type: ignore
from sklearn.preprocessing import StandardScaler, MinMaxScaler# type: ignore
from sklearn.model_selection import GridSearchCV, StratifiedKFold # type: ignore
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score, roc_curve, precision_recall_curve, make_scorer# type: ignore
import tensorflow as tf                                 # type: ignore
from tensorflow import keras                            # type: ignore
from keras.layers import Dense, Dropout                 # type: ignore
from keras.optimizers import Adam, SGD                  # type: ignore
from keras.callbacks import EarlyStopping               # type: ignore
from tensorboard.plugins.hparams import api as hp       # type: ignore
import matplotlib.pyplot as plt                         # type: ignore
from imblearn.over_sampling import RandomOverSampler, SMOTE# type: ignore
from imblearn.under_sampling import TomekLinks          # type: ignore
from itertools import combinations_with_replacement
warnings.filterwarnings(action='ignore')

tf.config.set_visible_devices([], 'GPU')

C:\Users\Hi\AppData\Local\Temp\ipykernel_4040\2470395882.py:14: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import Hyperband                 # type: ignore


In [3]:
# GPU 설정
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print(physical_devices[0])

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [9]:
WEIGHTED_RECOVERY_RATE = 0.44
RISK_FREE = 1.04
origin_train_df = pd.read_csv("train.csv")
origin_test_df = pd.read_csv("test.csv")
def get_expected_train_roe(y_pred, y_true):
    earning = np.sum(origin_train_df.loc[np.where((y_pred==0)&(y_true==0))]['total_loan_amnt'])
    recevored = np.sum(origin_train_df.loc[np.where((y_pred==0)&(y_true==1))]['total_loan_amnt'])*WEIGHTED_RECOVERY_RATE
    risk_free = np.sum(origin_train_df.loc[np.where(y_pred==1)]['total_loan_amnt'])*RISK_FREE
    return earning+recevored+risk_free

def get_expected_train_roe(y_pred, y_true):
    earning = np.sum(origin_test_df.loc[np.where((y_pred==0)&(y_true==0))]['total_loan_amnt'])
    recevored = np.sum(origin_test_df.loc[np.where((y_pred==0)&(y_true==1))]['total_loan_amnt'])*WEIGHTED_RECOVERY_RATE
    risk_free = np.sum(origin_test_df.loc[np.where(y_pred==1)]['total_loan_amnt'])*RISK_FREE
    return earning+recevored+risk_free

KeyError: 'total_loan_amnt'

In [3]:
RANDOM_STATE = 30

class DNNModel:
    def __init__(self) -> None:
        self.model = None
        self.df = None
        self.x_train = None
        self.y_train = None
        self.input_dim = 0
        self.model_name = ""

    def load_file(self, file_path:str)->None:
        self.df = pd.read_csv(file_path)
        x_train = self.df.drop(columns=['loan_status', 'Unnamed: 0'])
        # x_train = self.df.drop(columns=['loan_status'])
        
        y_train = self.df['loan_status']
        nan_indices = np.isnan(x_train).any(axis=1)
        self.x_train = x_train[~nan_indices]
        self.y_train = y_train[~nan_indices]
    
    def __get_n_components_from_pca(self, scaled_data:pd.DataFrame, threshold:float) -> int:
        pca = PCA()
        pca.fit(scaled_data)
        explained_variance = pca.explained_variance_ratio_
        cumulative_explained_variance = np.cumsum(explained_variance)
        n_components = np.argmax(cumulative_explained_variance >= threshold) + 1
        return n_components
    
    def scaling_data(
            self, 
            scaler:MinMaxScaler|StandardScaler=StandardScaler, 
            threshold:float=0.95, 
            over_sampler:SMOTE|RandomOverSampler=None, 
            under_sampler:TomekLinks=None)->None:
        x_scaled = scaler().fit_transform(self.x_train)
        self.input_dim = self.__get_n_components_from_pca(x_scaled, threshold=threshold)
        self.model_name+=(f'scaler({scaler.__name__})_pca({(int(threshold*100))})_input({str(self.input_dim)})')
        self.x_train = PCA(n_components=self.input_dim).fit_transform(x_scaled).astype('float32')
        if over_sampler!=None:
            os = over_sampler(random_state=RANDOM_STATE)
            self.x_train, self.y_train = os.fit_resample(self.x_train,self.y_train)
            self.model_name+=f'_sampler({os.__class__.__name__})'
            print(f"====Data set resampled(oversampled)_{os.__class__.__name__}")
        elif under_sampler!=None:
            us = under_sampler(random_state=RANDOM_STATE, sampling_strategy='majority')
            self.x_train, self.y_train = us.fit_resample(self.x_train, self.y_train)
            self.model_name+=f'_sampler({us.__class__.__name__})'
            print(f"====Data set resampled(undersampled)_{us.__class__.__name__}")
        print(f"==label ratio")
        print(f"True Label:\t{np.sum(self.y_train==1)/len(self.y_train)}")
        print(f"False Label:\t{np.sum(self.y_train==0)/len(self.y_train)}")
        self.x_train, self.x_validation, self.y_train, self.y_validation = train_test_split(self.x_train, self.y_train, test_size=0.2, stratify=self.y_train, random_state=RANDOM_STATE)
    
    def split_from_data(self):
        self.x_train, self.x_validation, self.y_train, self.y_validation = train_test_split(self.x_train, self.y_train, test_size=0.2, stratify=self.y_train, random_state=RANDOM_STATE)

    def __build_model(self, num_layers:int, num_nodes:int, loss:str, num_nodes_per_each_layers:list[int]=None)->keras.Sequential:
        # Define and compile the model
        model = keras.Sequential()
        model.add(Dense(num_nodes, input_dim=self.input_dim, activation='relu'))
        for layer_index in range(num_layers):
            if num_nodes_per_each_layers:
                model.add(Dense(num_nodes_per_each_layers[layer_index], activation='relu'))
            else:
                model.add(Dense(num_nodes, activation='relu'))
            model.add(Dropout(0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(optimizer='Adam', loss=loss, metrics=["accuracy"])
        return model
    
    def searching_best_param_grid(
            self, 
            grid_params:dict,
            params:dict={'min_delta':0.001, "n_splits":5},
            scoring='accuracy'
            ):
        kfold = StratifiedKFold(random_state=RANDOM_STATE,
                n_splits=params['n_splits']
                )
        model = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=self.__build_model)
        # GridSearchCV 생성
        grid_search = GridSearchCV(estimator = model,
                                param_grid = grid_params,
                                cv = kfold,
                                scoring=scoring)
        early_stopping = EarlyStopping(monitor='loss',min_delta=params['min_delta'])
        # GridSearchCV fit 시작
        grid_search.fit(self.x_train, self.y_train, callbacks=[early_stopping])
        # 최적의 param
        print(f"Best params: {grid_search.best_params_}")
        # 최적의 param일 경우 최적의 accuracy
        print(f"Best average accuracy: {grid_search.best_score_}")
        return grid_search.best_params_, grid_search.best_score_

    def build_model_with_best_params(
            self, 
            best_params:dict, 
            params:dict={'min_delta':0.001, "n_splits":5}, 
            threshold:float=0.5)->dict:
        valid_accs, valid_f1s, valid_recalls, valid_precisions = [], [], [], []
        result_dict = {
                       'average validation accuracy':0,
                       'average validation recall':0,
                       'average validation precision':0,
                       'average validation f1':0,
                       'validation accuracy':0,
                       'validation recall':0,
                       'validation precision':0,
                       'validation f1':0,
                       }
        y_train_reshaped = np.reshape(self.y_train,(-1))
        model = self.__build_model(best_params['num_layers'], best_params['num_nodes'], best_params['loss'])
        kf = StratifiedKFold(n_splits=params['n_splits'], shuffle=True, random_state=RANDOM_STATE)
        early_stopping = EarlyStopping(monitor='loss',min_delta=params['min_delta'])
        for train_index, val_index in kf.split(self.x_train, y_train_reshaped):
            X_train_fold, X_val_fold = self.x_train[train_index], self.x_train[val_index]
            Y_train_fold, Y_val_fold = y_train_reshaped[train_index], y_train_reshaped[val_index]
            # 모델 학습
            model.fit(X_train_fold, 
                    Y_train_fold, 
                    batch_size=best_params['batch_size'], 
                    epochs=best_params['epochs'], 
                    verbose=1,
                    callbacks=[early_stopping])
            # 모델 validation
            valid_loss, valid_acc= model.evaluate(X_val_fold, Y_val_fold)
            pred = model.predict(X_val_fold).flatten()
            pred = np.where(pred >= threshold, 1 , 0)
            valid_accs.append(valid_acc)
            valid_f1s.append(f1_score(Y_val_fold, pred))
            valid_precisions.append(precision_score(Y_val_fold, pred))
            valid_recalls.append(recall_score(Y_val_fold, pred))
        result_dict['average validation accuracy'] = np.mean(valid_accs)
        result_dict['average validation recall'] = np.mean(valid_recalls)
        result_dict['average validation precision'] = np.mean(valid_precisions)
        result_dict['average validation f1'] = np.mean(valid_f1s)
        print("###################################")
        print("avg Validation accuracy:", result_dict['average validation accuracy'])
        print("avg Validation recall:", result_dict['average validation recall'])
        print("avg Validation precision:", result_dict['average validation precision'])
        print("avg Validation F1-score:", result_dict['average validation f1'])

        valid_pred = model.predict(self.x_validation).flatten()
        valid_pred = np.where(valid_pred>=threshold, 1, 0)

        result_dict['validation accuracy'] = accuracy_score(Y_val_fold, pred)
        result_dict['validation f1'] = (f1_score(Y_val_fold, pred))
        result_dict['validation precision'] = (precision_score(Y_val_fold, pred))
        result_dict['validation recall'] = (recall_score(Y_val_fold, pred))
        print("###################################")
        print("Validation accuracy:", result_dict['validation accuracy'])
        print("Validation recall:", result_dict['validation recall'])
        print("Validation precision:", result_dict['validation precision'])
        print("Validation F1-score:", result_dict['validation f1'])
        self.model = model
        return result_dict
    
    def save_model(self, file_path:str="dnn_models/0425/dnn_")->None:
        self.model.save(file_path+self.model_name+'.h5')
 
    def load_model(self, model_file_path:str="dnn_models/0425/dnn_"):
        self.model = tf.keras.models.load_model(model_file_path)

    def get_input_dim(self)->int:
        return self.input_dim

    def get_model(self)->keras.Sequential:
        return self.model

    def get_model_name(self)->str:
        return self.model_name

    def get_roc_auc_score(self, save:bool=True)->float:
        valid_pred = self.model.predict(self.x_validation).flatten()
        fprs, tprs, thresholds = roc_curve(self.y_validation, valid_pred)
        plt.figure(figsize=(10, 5))
        plt.plot(fprs, tprs, label='ROC')
        plt.xlabel("FPR(Fall-Out)")
        plt.ylabel("TPR(Recall):재현률")
        plt.legend()
        if save:
            plt.savefig(f'plots/{self.model_name}.png')
        plt.show()
        return roc_auc_score(self.y_validation, valid_pred)
    
    def draw_precision_recall_curve_plot(self, min_delta:float=0.01, save:bool=True)->float:
        valid_pred = self.model.predict(self.x_validation).flatten()
        precisions, recalls, thresholds = precision_recall_curve(self.y_validation, valid_pred)
        # y축인 precisions와 recalls의 갯수 = threshold 갯수 + 1 이어서 x,y 갯수를 동일하게 맞춰줌
        thres_boundary = thresholds.shape[0]
        plt.plot(thresholds, precisions[:thres_boundary], "r", label="precision")
        plt.plot(thresholds, recalls[:thres_boundary], label="recall")

        # x축 스케일 0.1 단위로 조정
        start, end = plt.xlim()
        plt.xticks( np.round( np.arange(start, end, 0.1), 2))
        # 라벨링
        plt.xlabel("Threshold value"), plt.ylabel("Precision and Recall value")
        plt.legend()
        if save:
            plt.savefig(f'plots/{self.model_name}_precision&recall.png')
        plt.show()
        min_delta_index = None
        for i in range(len(precisions)):
            delta = abs(precisions[i] - recalls[i])
            if delta <= min_delta:
                min_delta_index = i
                break
        return recalls[min_delta_index]
    
    def predict_test(self, test_x, test_y, threshold):
        result_dict ={}
        y_pred = self.model.predict(test_x).flatten()
        y_pred = np.where(y_pred>=threshold, 1, 0)
        print("###################################")
        print("Validation accuracy:", accuracy_score(test_y, y_pred))
        print("Validation recall:", recall_score(test_y, y_pred))
        print("Validation precision:", precision_score(test_y, y_pred))
        print("Validation F1-score:", f1_score(test_y, y_pred))
        return y_pred
    
    def get_y_train(self)->np.ndarray:
        return self.y_train
    
    def get_y_validation(self)->np.ndarray:
        return self.y_validation

In [5]:
train_df = pd.read_csv("data/modified_0420.csv")
train_df.drop(columns=['loan_status', 'Unnamed: 0'], inplace=True)
sc, pca = get_scaler_pca_with_origin_data(StandardScaler, n_comp=86, origin_df=train_df)

In [6]:
model_file_name = 'dnn_scaler(StandardScaler)_pca(100)_input(86)_sampler(SMOTE).h5'
dnn_model = DNNModel()
test_df =pd.read_csv('data/modified_test.csv')
dnn_model.load_model(model_file_path='dnn_models/0425/'+model_file_name)
n_comp = dnn_model.get_model().summary()
df =pd.read_csv('data/modified_0420.csv')
X = df.drop(columns=['Unnamed: 0', 'loan_status'])
Y = df['loan_status']
resampled_x, resampled_y = SMOTE(random_state=RANDOM_STATE).fit_resample(X, Y)
train_df = df.drop(columns=['loan_status', 'Unnamed: 0'])
sc, pca = get_scaler_pca_with_origin_data(StandardScaler, n_comp=86, origin_df=resampled_x)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 86)                7482      
                                                                 
 dense_6 (Dense)             (None, 86)                7482      
                                                                 
 dropout_3 (Dropout)         (None, 86)                0         
                                                                 
 dense_7 (Dense)             (None, 86)                7482      
                                                                 
 dropout_4 (Dropout)         (None, 86)                0         
                                                                 
 dense_8 (Dense)             (None, 86)                7482      
                                                                 
 dropout_5 (Dropout)         (None, 86)               

In [7]:
test_x = test_df.drop(columns=['loan_status', 'Unnamed: 0']).to_numpy()
test_y = test_df['loan_status'].to_numpy()
test_x = sc.transform(test_x)
test_x = pca.transform(test_x)

In [12]:
dnn_model.predict_test(test_x, test_y, threshold=0.45)

23584/23584 [==============================] - 18s 773us/step
###################################
Validation accuracy: 0.4782212328404092
Validation recall: 0.8480439087725741
Validation precision: 0.25868725096413886
Validation F1-score: 1.0


array([0, 0, 1, ..., 0, 1, 1])